In [1]:
import pandas as pd
import numpy as np
import csv

# For email sending 
import smtplib, ssl

## email.mime subclasses
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import time


# Email info

In [27]:
email_sender = 'facebookmarketbot@gmail.com'
password_sender = 'ncgohboyrqhypoke'
email_reciever = 'mai247@usf.edu'


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

# import time

from time import sleep

In [4]:
# Set up WEBDRIVER

# Set up Chrome Options
chrome_options = Options()
chrome_options.add_argument("start-maximized")


driver = webdriver.Chrome(executable_path="chromedriver.exe", options=chrome_options)

# Anh có thể chỉnh tất cả filter rồi qua block tiếp theo

c:\Users\minht\.conda\envs\WebScraping\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [5]:
# prepare the csv file to store the data
open('data.csv', 'w').close()

# write header to the csv file
with open('data.csv', 'a', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([ 'title','time', 'location','photo_link','post_link'])

# Search in each item and get info

In [6]:
# Go into each item and get the info
def search_info(items):
    # create a dataframe to store the data
    df = pd.DataFrame(columns=['title','time', 'location','photo_link','post_link'])

    for item in items:
        # photo link 
        photo_link = item.find_element(By.XPATH, './/div[@class="x1n2onr6"]//img').get_attribute('src')
        print(photo_link)

        # 3 things: title, price, location
        title_price_location = item.find_elements(By.XPATH, './/div[@class="x1gslohp xkh6y0r"]')
        price = title_price_location[0].text

        title = title_price_location[1].text
        # remove any character that is not a number or a letter
        title = ''.join(e for e in title if e.isalnum() or e == ' ')

        # if the title is longer than 20 characters, cut it down
        if len(title) > 20:
            title = title[:20] + "..."


        location = title_price_location[2].text

        print(price + " " + title + " " + location)

        # link to the item
        item_link = item.find_element(By.XPATH, './/a').get_attribute('href')
        print(item_link)

        print(" ----------------- ") 

        # write to the csv file
        with open('data.csv', 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([title, price, location, photo_link, item_link])

        # add to the dataframe
        df = df.append({'title': title, 'time': price, 'location': location, 'photo_link': photo_link, 'post_link': item_link}, ignore_index=True)

    # return the dataframe
    return df

        


# Auto Scroll

In [81]:
def scroll(time):
    # Scroll down to load more items 10000 times
    for i in range(time):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(1)

    # Now loop through all items and get the info


# Core

In [7]:
def Core():
        
        # just for the waiting
        WebDriverWait(driver, 5).\
                until(EC.presence_of_element_located((By.XPATH,'//div[@class="xkrivgy x1gryazu x1n2onr6"]'))).text

        items = driver.find_elements(By.XPATH, '//div[@class="x9f619 x78zum5 x1r8uery xdt5ytf x1iyjqo2 xs83m0k x1e558r4 x150jy0e xnpuxes x291uyu x1uepa24 x1iorvi4 xjkvuk6"]//div[@class="xjp7ctv"]')
        print(len(items))

        # Now scrap data
        df =  search_info(items)

        return df



In [8]:
# get all items
item = "PS5"
url = f"https://www.facebook.com/marketplace/tampa/search/?daysSinceListed=1&query={item}&exact=false"

# go to the url
driver.get(url)

# Generate the HTML For email

In [9]:
def GenerateHTML(new_item_list):
    html = """
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="utf-8">
        <title>New Items</title>
        <style>
        body {
            font-family: Arial, sans-serif;
            font-size: 16px;
            line-height: 1.5;
            background-color: #f4f4f4;
        }

        .card {
            width: 300px;
            height: 400px;
            border: 1px solid #ccc;
            border-radius: 5px;
            overflow: hidden;
            display: inline-block;
            margin-right: 20px;
            margin-bottom: 20px;
            float: left;
        }

        .card img {
            width: 100%;
            height: 60%;
            object-fit: cover;
        }

        .card-body {
            padding: 10px;
            text-align: left;
        }

        .card-title {
            font-size: 20px;
            font-weight: bold;
            margin: 0 0 10px;
            text-align: left;
        }

        .card-text {
            margin: 3px 0;
            text-align: left;
        }

        .card-button {
            display: block;
            margin-top: 10px;
            padding: 5px 10px;
            border: 1px solid #ccc;
            border-radius: 5px;
            background-color: #fff;
            color: #333;
            text-align: center;
            text-decoration: none;
        }

        .card-button:hover {
            background-color: #f4f4f4;
        }
        </style>
    </head>
    <body>
        <h2>Featured Items</h2>
    """

    for item in new_item_list:
        html += f"""
        <div class="card">
        <img src="{item['photo_link']}" alt="{item['title']}">
        <div class="card-body">
            <h3 class="card-title">{item['title']}</h3>
            <p class="card-text">Price: {item['time']}</p>
            <p class="card-text">Location: {item['location']}</p>
            <a href="{item['post_link']}" class="card-button">Go to Post</a>
        </div>
        </div>
        """

    html += """
        </body>
        </html>
        """
    
    return html



In [28]:
def SendNotification(new_item):
    # generate the content of the email
    html = GenerateHTML(new_item)

    subject = 'New items on Facebook Marketplace'

    em = MIMEMultipart()
    em.attach(MIMEText(html, 'html'))


    em['From'] = email_sender
    em['To'] = email_reciever
    em['Subject'] = subject

    # connect to server
    context = ssl.create_default_context()

    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as server:
        server.login(email_sender, password_sender)
        server.sendmail(email_sender, email_reciever, em.as_string())


# Check for new items

In [11]:
df = Core()

while True:
    # get data and compare to see any new item
    df_new = Core()

    # compare the data
    if df.equals(df_new):
        print("No new item")
    else:
        print("New item")

        # extract the new item
        new_item = df_new[~df_new.isin(df)].dropna()

        # convert to a list of dictionary
        new_item = new_item.to_dict('records')

        # now send the notification to the user
        SendNotification(new_item)

        # update the data
        df = df_new

    # sleep for 30 seconds
    time.sleep(1)
    

    

22
https://scontent.ftpf1-2.fna.fbcdn.net/v/t45.5328-4/333859589_6301637909868259_8954945492046797948_n.jpg?stp=c0.43.261.261a_dst-jpg_p261x260&_nc_cat=110&ccb=1-7&_nc_sid=c48759&_nc_ohc=lG1wdkKUgN8AX-Cfzby&_nc_ht=scontent.ftpf1-2.fna&oh=00_AfALONUoY1oR-EbpuHfhUj2rdr--dlr2cRokcnEkDVCQNQ&oe=640D7986
$450 Ps5 Disc Edition USE... Spring Hill, FL
https://www.facebook.com/marketplace/item/526305042909345/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
 ----------------- 
https://scontent.ftpf1-2.fna.fbcdn.net/v/t45.5328-4/334531657_5669760759813926_966119473081695655_n.jpg?stp=c0.0.261.261a_dst-jpg_p261x260&_nc_cat=101&ccb=1-7&_nc_sid=c48759&_nc_ohc=zLGqSHTzkjsAX_XnifL&_nc_ht=scontent.ftpf1-2.fna&oh=00_AfARtML2lfML-wqNqvt-0nOXP32czAh6DOtS6jUg-a_RTA&oe=640C40FC
$450 New Ps5 Digital Edit... Clearwater, FL
https://www.facebook.com/marketplace/item/225099203307654/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
 ----------------- 
https://scontent.ftpf1

KeyboardInterrupt: 

In [10]:
driver.quit()